**实验目标：**

通过本实验，你将深入了解和实践说话人识别技术，并掌握利用声音特征进行有效说话人识别的基本方法，了解不同特征和模型对识别准确率的影响。

实验的核心目标是使用TIMIT数据集来训练一个说话人识别系统，涵盖数据预处理、特征提取、模型训练和评估等关键步骤。


**实验方法：**

**1. 数据预处理和划分(可选)：**
  - 数据集下载地址（4月17日前有效）：https://f.ws59.cn/f/du8yd2536vl
  - 为了方便大家，我们提供了划分好的TIMIT数据集结构，当然你也可以根据需求自行划分该数据集。
  - 为简化难度，我们排除了SA的两个方言句子，并在剩余的8个句子中选取了SX的5个句子和SI的1个句子作为训练集，SI的另外2个句子作为测试集。
  - 该链接下载的数据集只保留了音频文件，完整数据集（包含音频对应文本、标注等信息）可参见备注链接下载。
  
**2. 特征提取：**
  - 学习并实现包括但不限于MFCC特征等特征的提取，探索声音信号的频率和时间特性。
  - 鼓励尝试和比较其他特征提取方法，例如LPCC或声谱图特征，以理解不同特征对识别性能的影响。
  
**3. 模型选择和训练：**
  - 探索并选择适合的分类器和模型进行说话人识别，如GMM、Softmax分类器或深度学习模型。
  - 实现模型训练流程，使用训练集数据训练模型。
  
**4. 评估和分析：**
  - 使用准确率作为主要的评价指标在测试集上评估模型性能。
  - 对比不同特征和模型的性能，分析其对说话人识别准确率的影响。
  - 可视化不同模型的识别结果和错误率，讨论可能的改进方法。

**实验要求：**
  - 1.选择并实现至少一种特征的提取，并鼓励尝试其他特征提取方法。
  - 2.选择并实现至少一种分类器或模型进行说话人识别，并使用准确率评估指标评估其性能。
  - 3.通过实验对比、分析和可视化，撰写详细的实验报告，包括实验目的、实验方法、结果分析和结论。
  - 4.实验报告应以清晰、逻辑性强的形式呈现，图表和结果应清楚明了。

**其他说明：**
  - 实验的最终打分环节会看识别性能，会对原理和实现代码部分做抽查提问，综合评定成绩。
  - 我们**鼓励做原创性探索**，即使性能不是很好，但有创新性、有价值、有意义的探索和尝试会有额外加分。
  - 原数完整据集下载地址：https://drive.google.com/file/d/180mSIiXN9RVDV2Xn1xcWNkMRm5J5MjN4/view?usp=sharing \
    或国内访问地址（4月17日前有效）：https://f.ws59.cn/f/du8xu130kba

## 1. 实验准备

In [4]:
## 导入必要的库
import os
from tqdm import tqdm

import librosa
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
# 可以根据需要导入其他库，比如librosa用于音频处理

In [5]:
## 请从如下地址下载数据集（4月17日前有效）：https://f.ws59.cn/f/du8yd2536vl
# 数据集基本信息如下
# 方言地区：DR1～DR8
# 性别：F/M
# 说话者ID：3大写字母+1阿拉伯数字
# 句子ID：句子类型（SA/SI/SX）+编号
# 详细介绍参见 https://blog.csdn.net/qq_39373179/article/details/103788208

# 上述链接下载的数据集已经
TrainDir = "Dataset/TRAIN"
TestDir = "Dataset/TEST"
## 请在这里写代码加载我们划分好的TIMIT训练集和测试集。或者原始完整版数据集。


def pad_or_truncate(array, max_length):
    """;"""
    if array.shape[1] < max_length:
        pad_size = max_length - array.shape[1]
        array = np.pad(
            array, ((0, 0), (0, pad_size)), mode="constant", constant_values=(0, 0)
        )
    else:
        array = array[:, :max_length]
    return array


class trainset(Dataset):
    def __init__(self, dir, mode=0):
        self.mode = mode
        self.dir_path = dir
        self.drs = os.listdir(self.dir_path)
        self.wavs = dict()  # wav 文件名：标注
        self.label_to_index = {}  # 字典存储标签和索引
        self.current_index = 0

        # 初始化字典并填充 wav 列表
        for dr in self.drs:
            speakers = os.listdir(os.path.join(self.dir_path, dr))
            for s in speakers:
                wavs = os.listdir(os.path.join(self.dir_path, dr, s))
                for w in wavs:
                    if w.split(".")[-1] == "WAV":
                        label = dr + s[:1]  # 创建标签
                        if label not in self.label_to_index:
                            self.label_to_index[label] = self.current_index
                            self.current_index += 1
                        self.wavs[os.path.join(self.dir_path, dr, s, w)] = label

    def __getitem__(self, index):
        wav = list(self.wavs.keys())[index]
        label = self.wavs[wav]
        # 获取标签对应的索引
        label_index = self.label_to_index[label]
        # 将标签索引转换为张量
        label_tensor = torch.tensor(label_index, dtype=torch.long).to(device)
        waveform, sample_rate = torchaudio.load(wav)

        if self.mode == 0:
            mfcc = librosa.feature.mfcc(
                y=waveform.numpy()[0], sr=sample_rate, n_mfcc=13
            )
            mfcc_db = librosa.power_to_db(mfcc, ref=np.max)
            max_length = 128  # 这个值可以根据数据集中最长的样本来设置
            mfcc_db = pad_or_truncate(mfcc_db, max_length)

            mfcc_db = torch.tensor(mfcc).float().to(device)
            return mfcc_db, label_tensor
        else:
            mel_spec = librosa.feature.melspectrogram(
                y=waveform.numpy()[0], sr=sample_rate
            )
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
            max_length = 128  # 这个值可以根据数据集中最长的样本来设置
            mel_spec_db = pad_or_truncate(mel_spec_db, max_length)

            mel_spec_db = torch.tensor(mel_spec_db).float().to(device)

            return mel_spec_db, label_tensor

    def __len__(self):
        return len(self.wavs)

    def target(self):
        return len(self.label_to_index)


train_dataset = trainset(TrainDir, 1)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataset = trainset(TestDir, 1)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)
target = train_dataset.target()



## 3. 特征提取

In [6]:
## 请编写或使用库函数提取MFCC等音频特征


## 4. 模型选择和训练

In [7]:
## 在这部分，你可以选择不同的分类器和模型如GMM模型来进行实验
class AudioLSTM(nn.Module):
    def __init__(self, target):
        super(AudioLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=13, hidden_size=128, num_layers=2, batch_first=True
        )
        self.fc = nn.Linear(128, out_features=target)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x


class AudioCNN(nn.Module):
    def __init__(self, target):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
        output_size = 28800  # Adjusted calculation
        self.fc1 = nn.Linear(output_size, 100)
        self.fc2 = nn.Linear(100, target)

    def forward(self, x):
        x = x.unsqueeze(1)  # Adds a channel dimension
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = AudioCNN(target=target).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train_model(num_epochs):
    for epoch in range(num_epochs):
        loop = tqdm(train_loader, leave=True)
        for mfccs, labels in loop:
            optimizer.zero_grad()
            outputs = model(mfccs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            loop.set_postfix(loss=loss.item())
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")


train_model(10)

torch.save(model.state_dict(), "model_state_dict.pth")

Epoch [1/10]: 100%|██████████| 47/47 [01:51<00:00,  2.37s/it, loss=2.7] 


Epoch 1, Loss: 2.7009382247924805


Epoch [2/10]:  81%|████████  | 38/47 [00:35<00:08,  1.09it/s, loss=2.66]

## 5. 评价指标(准确率Accuracy)

In [ ]:
## 请编写代码或使用库函数accuracy_score计算测试集上的准确率Accuracy

model = AudioCNN(target=target)
model.load_state_dict(torch.load("model_state_dict.pth")).to(device)


def test_model():
    model.eval()

    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for mfccs, labels in test_loader:
            outputs = model(mfccs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # 计算正确的数量
            _, predicted = torch.max(outputs.data, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # 计算平均损失和准确度
    avg_loss = total_loss / len(test_loader)
    accuracy = total_correct / total_samples * 100

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")


test_model()

##  6. 分析和可视化

In [ ]:
## 请使用matplotlib等可视化库对你的实验结果进行可视化分析。
## 包括但不限于准确率的对比、错误分类的分析、特征的影响等。


## 7. 结果讨论
讨论你的模型性能，尝试解释为什么某些模型比其他模型表现好，以及可能的改进方法。

## 8. 保存模型（可选）
如果需要，可以在这里添加代码保存你的模型。